In [4]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, FloatVector, ListVector
import pandas as pd
from subprocess import call

In [17]:
df1_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/ASE/nano_merge_signal_eqtl.csv"
df1_trait_col = ""
df1_trait = "ASE"

df2_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/m6A/nano_merge_signal_m6Aqtl.csv"
df2_trait_col = "m6A_pos_1base"
df2_trait = "m6A"

gene_bed_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/stability/gene_merge.bed"
outdir = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/colco"


In [8]:
## 1、传入两个QTL的结果 筛选出所有fdr<0.1的点
merge_columns = ['chrom', 'strand', 'snp_pos_1base', 'rsID', 'A1', 'A2', 'MAF']
df1_column_names = merge_columns + ([df1_trait_col] if df1_trait_col else []) + ['p_value', 'fdr']
df2_column_names = merge_columns + ([df2_trait_col] if df2_trait_col else []) + ['p_value', 'fdr']

df1 = pd.read_csv(df1_file, usecols = df1_column_names)
df1 = df1[df1['fdr'] < 0.1]
df2 = pd.read_csv(df2_file, usecols = df2_column_names)
df2 = df2[df2['fdr'] < 0.1]

del df1['fdr'],df2['fdr']

df1 = df1.rename(columns={'p_value': f'p_value_{df1_trait}'})
df2 = df2.rename(columns={'p_value': f'p_value_{df2_trait}'})

## 2、筛选出所有重叠的QTL位点，然后对这些SNP位点进行gene注释
merged_df = pd.merge(df1, df2, on=merge_columns)
merged_df_anno = merged_df.drop_duplicates(subset=merge_columns)
merged_df_anno['snp_pos_0base'] = merged_df_anno['snp_pos_1base']-1
merged_df_anno['score'] = 0
merged_df_anno = merged_df_anno[['chrom','snp_pos_0base','snp_pos_1base','score','rsID','strand']]
merged_df_anno.to_csv(f"{outdir}/merged_df_anno.bed", sep="\t", index=False, header=False)
call(f'bedtools intersect -a {outdir}/merged_df_anno.bed -b {gene_bed_file} -s -wa -wb > {outdir}/merged_df_gene_anno.bed',shell=True)
merged_df_anno = pd.read_csv(f'{outdir}/merged_df_gene_anno.bed', sep="\t", header=None,usecols = [0,2,4,5,9])
merged_df_anno.columns = ['chrom','snp_pos_1base','rsID','strand','geneID']
merged_df = pd.merge(merged_df, merged_df_anno[['geneID'] + ['chrom', 'snp_pos_1base', 'rsID', 'strand']], on=['chrom', 'snp_pos_1base', 'rsID', 'strand'], how='left')
call(f'rm {outdir}/merged_df_anno.bed {outdir}/merged_df_gene_anno.bed',shell=True)

## 3、将这些SNP按照gene分类，如果gene中这种QTL含量较少 就合并分析区间，得到每个区间的QTL信息


In [ ]:
假设我现在想分析mQTL和m6AQTL之间的共定位关系 我是应该以甲基化位点+-100kb为分析区间还是以m6A位点+-100kb为分析区间呢？不同区间的选择是否会对最后的结果有影响

In [ ]:
import numpy as np
import statsmodels.api as sm

def calculate_se_manual(X, y):
    # 添加常数项
    X = sm.add_constant(X)
    
    # 拟合模型
    model = sm.OLS(y, X).fit()
    
    # 预测值
    y_pred = model.predict(X)
    
    # 计算残差平方和
    rss = np.sum((y - y_pred)**2)
    
    # 计算自变量的离差平方和
    x_demeaned = X[:, 1] - np.mean(X[:, 1])
    ssx = np.sum(x_demeaned**2)
    
    # 计算误差方差的估计
    n = len(y)
    s_squared = rss / (n - 2)
    
    # 计算SE
    se = np.sqrt(s_squared / ssx)
    
    return se

# 使用statsmodels直接获取SE
def calculate_se_statsmodels(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model.bse[1]  # 返回斜率的SE

# 示例使用
X = np.random.rand(100, 1)
y = 2 * X + 1 + np.random.randn(100, 1) * 0.1

se_manual = calculate_se_manual(X, y)
se_statsmodels = calculate_se_statsmodels(X, y)

print(f"Manually calculated SE: {se_manual[0]}")
print(f"Statsmodels SE: {se_statsmodels}")


df[['p_value','beta', 'SE']] = results

In [ ]:
## 4、对所有的区间进行共定位分析，这里注意会存在有的SNP同时影响多种表型

In [ ]:
## 5、整理出所有区间中PH4>0.75的SNP

In [ ]:
input <- merge(eqtl, gwas, by="rs_id", all=FALSE, suffixes=c("_eqtl","_gwas"))
result <- coloc.abf(dataset1=list(pvalues=input$pval_nominal_gwas, type="cc", s=0.33, N=50000), dataset2=list(pvalues=input$pval_nominal_eqtl, type="quant", N=10000), MAF=input$maf)

need_result=result$results %>% filter(SNP.PP.H4 > 0.95)

In [ ]:
# 启用pandas和R的数据框之间的转换
pandas2ri.activate()

# 加载R包
coloc = importr('coloc')

# 创建pandas数据框
data1 = pd.DataFrame({
    'snp': ['rs1', 'rs2', 'rs3'],
    'beta': [0.2, 0.4, 0.6],
    'varbeta': [0.02, 0.04, 0.06],
    'MAF': [0.1, 0.2, 0.3],
    'N': [1000, 1000, 1000],
    'type': ['quant', 'quant', 'quant']
})

data2 = pd.DataFrame({
    'snp': ['rs1', 'rs2', 'rs3'],
    'beta': [0.3, 0.5, 0.7],
    'varbeta': [0.03, 0.05, 0.07],
    'MAF': [0.1, 0.2, 0.3],
    'N': [1000, 1000, 1000],
    'type': ['quant', 'quant', 'quant']
})

# 将pandas数据框转换为R数据框
rdf1 = pandas2ri.py2rpy(data1)
rdf2 = pandas2ri.py2rpy(data2)

# 运行coloc.abf函数进行共定位分析
result = coloc.coloc_abf(dataset1=rdf1, dataset2=rdf2)

# 将结果转换回pandas数据框
result_df = pandas2ri.rpy2py(result)
print(result_df)


In [ ]:
# 假设你有一个输入字典input包含pval_nominal_gwas, pval_nominal_eqtl, 和maf
input_data = {
    'pval_nominal_gwas': [0.01, 0.02, 0.05],  # 示例数据
    'pval_nominal_eqtl': [0.03, 0.04, 0.01],  # 示例数据
    'maf': [0.1, 0.2, 0.3]                    # 示例数据
}

# 将输入数据转换为R的向量
pval_nominal_gwas = FloatVector(input_data['pval_nominal_gwas'])
pval_nominal_eqtl = FloatVector(input_data['pval_nominal_eqtl'])
maf = FloatVector(input_data['maf'])

# 创建dataset1和dataset2
dataset1 = ListVector({
    'pvalues': pval_nominal_gwas,
    'type': 'cc',
    's': 0.33,
    'N': 50000
})

dataset2 = ListVector({
    'pvalues': pval_nominal_eqtl,
    'type': 'quant',
    'N': 10000
})

# 运行coloc.abf函数
result = coloc.coloc_abf(dataset1=dataset1, dataset2=dataset2, MAF=maf)

# 将结果转换为Python数据结构（如需要）
result_df = pandas2ri.rpy2py(result)
print(result_df)
